In [ ]:
#default_exp AbstractMethod

In [ ]:
#export
#hide
from typing import Union, List

import sys
sys.path.append("..")

from hephaestus.EditOperations import *

In [ ]:
#hide
from nbdev.showdoc import *

# AbstractMethod

> Defines the AbstractMethod class which represents a token-abstracted Java method.

In [ ]:
#export
class AbstractMethod:
    """
    Creates an AbstractMethod from the given `tokens`, which can be either:

    - a string with tokens delimited by `delimiter` (defaults to a single space)
    - a list of tokens

    Note that empty tokens are ignored.
    """

    def __init__(self, tokens: Union[str, List[str]], delimiter: str = " ") -> None:

        if type(tokens) is str:
            self.__tokens = [] if len(tokens) == 0 else tokens.split(delimiter)
        else:
            self.__tokens = tokens.copy()
        
        # remove empty tokens
        self.__tokens = [token for token in self.__tokens if token != ""]
    
    def __eq__(self, other: "AbstractMethod") -> bool:
        return type(other) is AbstractMethod and self.__tokens == other.__tokens
    
    def __getitem__(self, key: int) -> str:
        return self.__tokens[key]
    
    def __len__(self) -> int:
        return len(self.__tokens)

    def __str__(self) -> str:
        return repr(" ".join(self.__tokens))[1:-1]

    def __repr__(self) -> str:
        return str(self)
    
    def applyEditOperation(self, operation: EditOperation):
        """
        Applies the given `operation`.
        """
        operation.applyToTokens(self.__tokens)

    def applyEditOperations(self, operations: List[EditOperation]):
        """
        Applies the given list of `operations` in order.
        """
        for op in operations:
            self.applyEditOperation(op)
    
    def getEditOperationsTo(self, other: "AbstractMethod") -> List[Union[InsertOperation, DeleteOperation, ReplaceOperation]]:
        """
        Returns the minimal list of basic edit operations (no CompoundOperations), which if applied, would
        result in the `AbstractMethod` given by `other`. The length of the returned list is the Levenshtein
        distance to `other`.
        """
        
        matrix = self.__getEditOpsMatrix(other)
        editOps = []

        r = len(matrix) - 1
        c = len(matrix[0]) - 1

        while True:
            if matrix[r][c] == 0:
                break
            elif r == 0:
                c -= 1
                editOps.insert(0, InsertOperation(c, other[c]))
            elif c == 0:
                r -= 1
                editOps.insert(0, DeleteOperation(c))
            elif self[r - 1] == other[c - 1]:
                r -= 1
                c -= 1
            elif matrix[r][c] == matrix[r - 1][c - 1] + 1:
                r -= 1
                c -= 1
                editOps.insert(0, ReplaceOperation(c, other[c]))
            elif matrix[r][c] == matrix[r][c - 1] + 1:
                c -= 1
                editOps.insert(0, InsertOperation(c, other[c]))
            elif matrix[r][c] == matrix[r - 1][c] + 1:
                r -= 1
                editOps.insert(0, DeleteOperation(c))
            else:
                raise RuntimeError("AbstractMethod: invalid matrix!")
        
        return editOps

    def __getEditOpsMatrix(self, other: "AbstractMethod") -> List[List[int]]:

        numRows = len(self) + 1
        numCols = len(other) + 1

        # initialize matrix
        matrix = []
        for r in range(numRows):
            matrix.append([c if r == 0 else 0 for c in range(numCols)])
            matrix[r][0] = r
        
        # iterate through matrix and assign values
        for r in range(1, numRows):
            for c in range(1, numCols):

                left =    matrix[r    ][c - 1]
                topLeft = matrix[r - 1][c - 1]
                top =     matrix[r - 1][c    ]

                if self[r - 1] == other[c - 1]:
                    matrix[r][c] = topLeft
                else:
                    matrix[r][c] = min(left, topLeft, top) + 1

        return matrix

In [ ]:
method1 = AbstractMethod("private static int METHOD_1 ( ) { return 0 ; }")
method1

private static int METHOD_1 ( ) { return 0 ; }

In [ ]:
method2 = AbstractMethod("public double METHOD_1 ( double VAR_1 ) { return VAR_1 ; }")
method2

public double METHOD_1 ( double VAR_1 ) { return VAR_1 ; }

## Interact with Edit Operations

In [ ]:
#hide_input
show_doc(AbstractMethod.getEditOperationsTo)

<h4 id="AbstractMethod.getEditOperationsTo" class="doc_header"><code>AbstractMethod.getEditOperationsTo</code><a href="__main__.py#L50" class="source_link" style="float:right">[source]</a></h4>

> <code>AbstractMethod.getEditOperationsTo</code>(**`other`**:[`AbstractMethod`](/hephaestus/AbstractMethod.html))

Returns the minimal list of basic edit operations (no CompoundOperations), which if applied, would
result in the [`AbstractMethod`](/hephaestus/AbstractMethod.html) given by `other`. The length of the returned list is the Levenshtein
distance to `other`.

In [ ]:
method1 = AbstractMethod("private static int METHOD_1 ( ) { return 0 ; }")
method2 = AbstractMethod("public double METHOD_1 ( double VAR_1 ) { return VAR_1 ; }")

operations = method1.getEditOperationsTo(method2)

operations

[DELETE 0,
 REPLACE 0 -> 'public',
 REPLACE 1 -> 'double',
 INSERT 4 -> 'double',
 INSERT 5 -> 'VAR_1',
 REPLACE 9 -> 'VAR_1']

In [ ]:
#hide_input
show_doc(AbstractMethod.applyEditOperation)

<h4 id="AbstractMethod.applyEditOperation" class="doc_header"><code>AbstractMethod.applyEditOperation</code><a href="__main__.py#L37" class="source_link" style="float:right">[source]</a></h4>

> <code>AbstractMethod.applyEditOperation</code>(**`operation`**:`Union`\[[`InsertOperation`](/hephaestus/EditOperations.html#InsertOperation), [`DeleteOperation`](/hephaestus/EditOperations.html#DeleteOperation), [`ReplaceOperation`](/hephaestus/EditOperations.html#ReplaceOperation), [`CompoundOperation`](/hephaestus/EditOperations.html#CompoundOperation)\])

Applies the given `operation`.

> Note: This changes the original `AbstractMethod`, so you should make a copy if you want to keep the original.

In [ ]:
#hide_input
show_doc(AbstractMethod.applyEditOperations)

<h4 id="AbstractMethod.applyEditOperations" class="doc_header"><code>AbstractMethod.applyEditOperations</code><a href="__main__.py#L43" class="source_link" style="float:right">[source]</a></h4>

> <code>AbstractMethod.applyEditOperations</code>(**`operations`**:`List`\[`Union`\[[`InsertOperation`](/hephaestus/EditOperations.html#InsertOperation), [`DeleteOperation`](/hephaestus/EditOperations.html#DeleteOperation), [`ReplaceOperation`](/hephaestus/EditOperations.html#ReplaceOperation), [`CompoundOperation`](/hephaestus/EditOperations.html#CompoundOperation)\]\])

Applies the given list of `operations` in order.

> Note: This changes the original `AbstractMethod`, so you should make a copy if you want to keep the original.

In [ ]:
method1 = AbstractMethod("private static int METHOD_1 ( ) { return 0 ; }")
method2 = AbstractMethod("public double METHOD_1 ( double VAR_1 ) { return VAR_1 ; }")

operations = method1.getEditOperationsTo(method2)
method1.applyEditOperations(operations)

In [ ]:
method1

public double METHOD_1 ( double VAR_1 ) { return VAR_1 ; }

In [ ]:
method2

public double METHOD_1 ( double VAR_1 ) { return VAR_1 ; }

In [ ]:
method1 == method2

True